In [1]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('best.pt')  # load a .pt file

# Use the model
# model.train(data="config.yml", epochs=100, amp=False)  # train the model for GPU
# model.train(data="config.yml", epochs=50)  # train the model for CPU
# metrics = model.val()  # evaluate model performance on the validation set


In [2]:
import cv2
import numpy as np
from detect import Coords
from PIL import Image
imgPath = ('./test.jpg')
# imgPath = ('./data/test/images/00047.jpg')
results = model(imgPath)
coords: list[Coords] = list()
for result in results:
    # print(result.boxes)
    for box in result.boxes:
        for xyxy in box.xyxy:
            coord = Coords(xyxy[0], xyxy[1], xyxy[2], xyxy[3])
            coords.append(coord)
    # im_array = result.plot()  # plot a BGR numpy array of predictions
    # im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    # # im.show()  # show image
    # im.save('results.jpg') 
# print(result["boxes"])
# import cv2
# print(metrics)
# path = model.export(format="onnx") # export the model to ONNX format
# print(path)
# results = model(img)  # predict on an image
# print(results)

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [3]:
for coord in coords:
    coord.drawBoundingBox(imgPath, saveImagePath='draw1.jpg')

In [4]:
for coord in coords:
    # Crop image 
    coord.cropImage(imgPath, saveImagePath='crop.jpg')

In [17]:
from recog import Recog
recog = Recog()
recog.preprocess('crop.jpg', "binary.jpg")
recog.read('binary.jpg')
print(recog.result)


[]


In [21]:
text = ""
for result in recog.result:
    print(result[1])
    text += result[1] 

V1
0r - 20201 1955


In [22]:
text = text.replace(" ", "")
print(text)

V10r-202011955


In [23]:
from IPython.display import clear_output, Image, display, HTML
import cv2

# Read one frame from the camera for testing
video = cv2.VideoCapture(0)
_, frame = video.read()
video.release()

import re
from timeit import timeit
import math

extensions=re.findall(r"\\\*(\.\w*)", cv2.imread.__doc__)

def test(extension = ".jpeg"):
    try:
        totalTime=0
        numTry=3
        for _ in range(numTry):
            totalTime+=timeit(lambda: display(Image(data=cv2.imencode(extension, frame)[1])), number=1)
            clear_output(wait=True)
        return totalTime/numTry, extension

    except cv2.error as e: #usually "unsupported file type"
        return (math.inf, extension, e)
for x in sorted(
    [test(extension) for extension in extensions], key=lambda x: x[0]
): print(x)

In [26]:
import cv2
from IPython.display import display, Image
import matplotlib.pyplot as plt
from detect import TrainYOLO, Coords
from recog import Recog
import numpy as np
import time
import requests
import imutils
url = "http://192.168.35.25:8080/shot.jpg"
# video = cv2.VideoCapture(0)
display_handle=display(None, display_id=True)
model = TrainYOLO("best.pt")

# initial_frame = 
recog = Recog()
try:
    while True:
        img_resp = requests.get(url) 
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
        img = cv2.imdecode(img_arr, -1) 
        frame = imutils.resize(img, width=1000, height=1800) 
        # _, frame = video.read()
        # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imwrite('frame.jpg', frame)
        coords: list[Coords] = model.detect('frame.jpg', saveImagePath='draw1.jpg')
        if len(coords) > 0:
            for coord in coords:
                coord.drawBoundingBox('draw1.jpg', saveImagePath='draw1.jpg')
                coord.cropImage('draw1.jpg', saveImagePath='crop.jpg')
                recog.preprocess('crop.jpg', "binary.jpg")
                recog.read('binary.jpg')
                for result in recog.result:
                    print("result is " + result[1])
                    text += result[1]
                print("result from detection is" + text)
        # frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        time.sleep(0.3)
        
        # Display the resulting frame
        # bounding_box: list[Coords] = model.detect(frame)
        # if bounding_box: 
        #     bounding_box[0].drawBoundingBox(frame, saveImagePath='draw.jpg')
        # Recognize tag and display text
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    video.release()
    display_handle.update(None)

None


image 1/1 c:\Users\jin-yee.chan\Documents\TaxtDetection\frame.jpg: 384x640 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\jin-yee.chan\Documents\TaxtDetection\frame.jpg: 384x640 (no detections), 26.1ms
Speed: 4.0ms preprocess, 26.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\jin-yee.chan\Documents\TaxtDetection\frame.jpg: 384x640 (no detections), 34.1ms
Speed: 3.0ms preprocess, 34.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\jin-yee.chan\Documents\TaxtDetection\frame.jpg: 384x640 (no detections), 40.1ms
Speed: 2.0ms preprocess, 40.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\jin-yee.chan\Documents\TaxtDetection\frame.jpg: 384x640 (no detections), 168.6ms
Speed: 2.0ms preprocess, 168.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:

In [ ]:
cropedImgPath = './crop.jpg'


In [ ]:
import pytesseract
from PIL import Image
print(pytesseract.image_to_string(Image.open(cropedImgPath)))

In [ ]:
import easyocr
reader = easyocr.Reader(['en'])

In [ ]:
result = reader.readtext(x)
print(result)

In [ ]:
# from doctr.models import ocr_predictor
# from doctr.io import DocumentFile
# model = ocr_predictor('db_resnet50', 'crnn_mobilenet_v3_small', pretrained=True)


In [ ]:
# cropedImgPath = './crop.jpg'
# image = DocumentFile.from_images(cropedImgPath)
# out = model(image)
# print(out)

In [ ]:
# import pytesseract
# from PIL import Image
# print(pytesseract.image_to_string(Image.open(cropedImgPath)))